# Opensearch User Behavior Insights (UBI)

### This notebook covers the basics around setting up UBI, ingesting data using the UBI plugin, and setting up a basic UBI opensearch dashboard

**Information regarding UBI:**

https://opensearch.org/docs/latest/search-plugins/ubi/index

https://github.com/opensearch-project/user-behavior-insights

In [1]:
from aips import get_engine, set_engine
from aips.spark.dataframe import from_sql
from aips.spark import create_view_from_collection
import tqdm
import aips.indexer
import requests, json
set_engine("opensearch")
engine = get_engine("opensearch")

In [2]:
aips.indexer.build_collection(engine, "products")
aips.indexer.build_collection(engine, "signals")

Wiping "products" collection
Creating "products" collection
Loading Products
Schema: 
root
 |-- upc: string (nullable = true)
 |-- name: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- long_description: string (nullable = true)

<Response [200]>
{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}
Successfully written 48194 documents
Wiping "signals" collection
Creating "signals" collection
Loading data/retrotech/signals.csv
Schema: 
root
 |-- query_id: string (nullable = true)
 |-- user: string (nullable = true)
 |-- type: string (nullable = true)
 |-- target: string (nullable = true)
 |-- signal_time: timestamp (nullable = true)

<Response [200]>
{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}
Successfully written 2172605 documents


### **Step 1**: Install and configure the OpenSearch UBI plugin

To install UBI on an opensearch cluster, execute the following command on a node or during the building of an image. This command has already been run on the AIPS opensearch node.

**bin/opensearch-plugin install https://github.com/o19s/opensearch-ubi/releases/download/release-v0.0.12.1-os2.14.0/opensearch-ubi-plugin-v0.0.12.1-os2.14.0.zip --batch**

### **Step 2**: - Bulk ingesting historic signals

Historic user events and queries should be bulk ingested into the appropriate UBI collections.

Here we bulk write all AIPS queries into the `ubi_queries` collection.

In [3]:
def get_queries_dataframe():
    signals_collection = engine.get_collection("signals")
    create_view_from_collection(signals_collection, "signals")
    queries = from_sql("SELECT * FROM signals WHERE type = 'query'")
    queries_transformed = queries.rdd.map(lambda r: 
        (r["signal_time"], r["query_id"], r["user"], r["target"]))
    ubi_queries_dataframe = queries_transformed.toDF(
        ["timestamp", "query_id", "client_id", "user_query"])
    return ubi_queries_dataframe

In [4]:
def batch_ingest_queries():
    queries_collection = engine.create_collection("ubi_queries")
    ubi_queries_dataframe = get_queries_dataframe()
    queries_collection.write(ubi_queries_dataframe)
    return queries_collection

#This line commented as batch query ingestion is done in a 
#different manner later with the extension for examples sake.
#queries_collection = batch_ingest_queries()

Next we can index events into the `ubi_events` collection which is intended to hold all non-query signals

In [5]:
def get_events_dataframe():
    signals_collection = engine.get_collection("signals")
    products_collection = engine.get_collection("products")
    create_view_from_collection(signals_collection, "signals")
    create_view_from_collection(products_collection, "products")
    query = """SELECT REPLACE(type, '-', '_') AS action_name, query_id, user AS client_id,
                      signal_time AS timestamp, type AS message_type,
                      target AS target, p.name AS message
               FROM signals s
               LEFT JOIN products p ON s.target == p.upc
               WHERE type != 'query'"""
    events = from_sql(query)
    return events

In [6]:
def batch_ingest_signals():
    events_collection = engine.create_collection("ubi_events")
    ubi_events_dataframe = get_events_dataframe()
    events_collection.write(ubi_events_dataframe)
    return events_collection

events_collection = batch_ingest_signals()

Wiping "ubi_events" collection
Creating "ubi_events" collection
['query', 'query_id', 'signal_time', 'target', 'type', 'user', 'id']
['_text_', 'long_description', 'manufacturer', 'name', 'name_fuzzy', 'name_ngram', 'short_description', 'short_description_ngram', 'upc', 'id']
<Response [200]>
{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}
Successfully written 1447146 documents


### **Step 3**: Live logging of queries and events

Queries and events must be ingested correctly and with complete data into UBI for best results. UBI stores queries seperate from other events, each in their respective collection `ubi_queries` and `ubi_events`. Live signal data collection should be hooked into the appropriate places in your stack.

Logging event data is as simple as writing an event document directly to the ubi_events collection. 

In [7]:
def add_example_event_to_ubi():
    collection = "products"
    event_doc = {"action_name": "purchase", #This is a name of the type of event/action that occurred
                 "client_id": "uid_000001", #This is id of the user/session taking the action
                 "message_type": "one_click_buy", #An additional action type, used for further action grouping
                 "message": "Succeeded", #An optional message string for the event
                 "query_id": "qid_000001", #The id of the query that led to this action
                 "target": "pid_000001"} #Any string representing the target of the action. Normally a doc/item id?

    response = requests.post(f"http://opensearch-node1:9200/ubi_events/_doc?",
                             json=event_doc)
    display(response.json())

add_example_event_to_ubi()

{'_index': 'ubi_events',
 '_id': 'MNOyJpkBEHIwjVwYeLm1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 1447146,
 '_primary_term': 1}

Queries should be collected at query time utilizing the UBI extension request handler. Here is an example of ingesting query data by adding an `ubi` property to the `ext` object during a search request:

In [8]:
def execute_example_query_with_ubi():        
    collection = "products"
    query = "cable"
    ubi_extension_data = {"ubi": {"query_id": "qid_000001",
                                  "client_id": "cid_000001",
                                  "user_query": query}}
    search_request = {
        "query": {"query_string": {"query": query,
                                   "fields": ["name", "manufacturer",
                                              "long_description", "short_description"]}},
        "size": 11, 
        "fields": ["*"],
        "ext": ubi_extension_data
    }

    response = requests.post(f"http://opensearch-node1:9200/{collection}/_search?",
                             json=search_request)
    display(response.json())

execute_example_query_with_ubi()

{'took': 12,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1165, 'relation': 'eq'},
  'max_score': 7.091094,
  'hits': [{'_index': 'products',
    '_id': '50644304606',
    '_score': 7.091094,
    '_source': {'upc': '50644304606',
     'name': 'Monster Cable - TV Screen Cleaning Kit',
     'manufacturer': 'Monster Cable',
     'short_description': 'High-tech reusable MicroFiber cloth cleans your screen without scratching it',
     'long_description': "Keep your big screen in pristine condition with this complete monitor cleaning kit. The advanced formula removes dust and fingerprints without streaks or stains. The reusable MicroFiber cloth won't scratch your screen."},
    'fields': {'short_description': ['High-tech reusable MicroFiber cloth cleans your screen without scratching it'],
     'name': ['Monster Cable - TV Screen Cleaning Kit'],
     'upc': ['50644304606'],
     'long_description': ["Keep your big scr

Notice UBI information is returned on the search response object with at least the ubi signal id linking to the ingested query. 

The following code will load all query signals into UBI by simulating user searches. This serves as a batch import of data for examples sake. Batch importing should normally just be done by batch indexing query signals directly into `ubi_queries` directly as shown earlier.


In [ ]:
def execute_search(collection, signal, log=False):
    signal.pop("timestamp", None) #The timestamp of a query is the time of search and cannot be passed in
    request = {"query": signal["user_query"],
               "query_fields": ["name", "manufacturer",
                                "long_description", "short_description"],
               "return_fields": ["*"],
               "limit": 10,
               "ubi": signal | {"store_name": "aips_store"}}
    try:
        return collection.search(**request)
    except:
        pass

def search_and_log_all_query_signals():
    products_collection = engine.get_collection("products")
    ubi_queries_dataframe = get_queries_dataframe()
    for q in tqdm.tqdm(ubi_queries_dataframe.collect(), total=ubi_queries_dataframe.count()):
        execute_search(products_collection, q.asDict())

search_and_log_all_query_signals()

### Loading UBI queries and events into AIPS

If you wish to load in UBI queries/events from your Opensearch cluster to work with the book, you can do so with the following code

In [5]:
def load_ubi_events_as_aips_dataframe():
    ubi_events_collection = engine.get_collection("ubi_events")
    create_view_from_collection(ubi_events_collection, "ubi_events")
    events = from_sql("SELECT * FROM ubi_events")
    events_transformed = events.rdd.map(lambda r: 
        (r["timestamp"], r["query_id"], r["client_id"],
         r["message"], r["message_type"]))
    return events_transformed.toDF(["signal_time", "query_id", "user", "target", "type"])

def load_ubi_queries_as_aips_dataframe():
    ubi_queries_collection = engine.get_collection("ubi_queries")
    create_view_from_collection(ubi_queries_collection, "ubi_queries")
    queries = from_sql("SELECT * FROM ubi_queries")
    queries_transformed = queries.rdd.map(lambda r: 
        (r["timestamp"], r["query_id"], r["client_id"],
         r["user_query"], "query"))
    return queries_transformed.toDF(["signal_time", "query_id", "user", "target", "type"])

def create_signals_collection_with_ubi_data():
    signals_collection = engine.create_collection("signals")
    events = load_ubi_events_as_aips_dataframe()
    queries = load_ubi_queries_as_aips_dataframe()
    signals_collection.write(queries)
    signals_collection.write(events, overwrite=False)
    return signals_collection

signals_collection = create_signals_collection_with_ubi_data()

Wiping "signals" collection
Creating "signals" collection
['action_name', 'client_id', 'message', 'message_type', 'query_id', 'target', 'timestamp', 'id']
['client_id', 'query', 'query_id', 'query_response_hit_ids', 'query_response_id', 'timestamp', 'user_query', 'id']
<Response [200]>
{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}
Successfully written 1 documents
<Response [200]>
{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}
Successfully written 1447147 documents


### Creating and viewing the UBI Dashboard

The following code will import the default Dashboard objects. The dashboard can be viewed here

http://opensearch-aips:5601/app/dashboards


In [3]:
def import_ubi_dashboard():
    with open("./engines/opensearch/build/ubi-dashboard-objects.json", "rb") as f: 
        dashboard_objects = json.load(f)
    dashboard_ndjson = "\n".join(json.dumps(d) for d in dashboard_objects)
    response = requests.post(f"http://opensearch-dashboards:5601/api/saved_objects/_import?createNewCopies=true",
                            files={"file": ("request.ndjson", dashboard_ndjson)},
                            headers={"kbn-xsrf": "true",
                                     "osd-version": "2.14.0",
                                     "osd-xsrf": "osd-fetch"})
    display(response.json())

import_ubi_dashboard()

JSONDecodeError: Extra data: line 2 column 1 (char 22442)